### Запросы в базу для получения последних актуальных данных для прогноза КЧ

In [1]:
import pandas as pd
import pyodbc
import datetime

# Начальная дата периода, по которому сделаем предсказание
dataStart = str(datetime.date.today() - datetime.timedelta(days=6))
dataStart

'2022-04-23'

In [2]:
# Подключение к вьюшке
driver = r'FreeTDS'
server = r'pdc-lst-str-02.corp.suek.ru'
port = r'1433'
database = r'ces_tgk12' 
username = r'sibgenco.local\omima'
password = r'VxjNOJV]!3'
tds = r'8.0'

conn = pyodbc.connect(f'\
DRIVER={driver};\
SERVER={server};\
PORT={port};\
DATABASE={database};\
UID={username};\
PWD={password};\
TDS_VERSION={tds};')

In [3]:
# Представление вьюшки
pd.read_sql('SELECT * FROM INFORMATION_SCHEMA.TABLES;',
            conn)

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,Ces_Tgk12,ComDis,TradeRegions,VIEW
1,Ces_Tgk12,ComDis,TemperatureTown,VIEW
2,Ces_Tgk12,ComDis,Hydrology,VIEW
3,Ces_Tgk12,ComDis,RgePlanGenerationAmount,VIEW
4,Ces_Tgk12,ComDis,GtpPlanConsume,VIEW
5,Ces_Tgk12,ComDis,SellNorem,VIEW
6,Ces_Tgk12,ComDis,ControlledSection,VIEW
7,Ces_Tgk12,ComDis,GtpPlanValue,VIEW
8,Ces_Tgk12,ComDis,RegionConsumeForecast,VIEW
9,Ces_Tgk12,ComDis,RegionPlanConsume,VIEW


In [4]:
qwerty = f'SELECT *\
           FROM ComDis.ControlledSection WHERE Date>=convert (datetime,\'{dataStart} 0:00:00\')'  

GtpPlanValue = pd.read_sql(qwerty, conn)
display(GtpPlanValue.head())

,ControlledSectionId,SectionName,Date,ReportDate,PlannedPowerFlow,PmaxFlow
0,4000001,Казахстан - Сибирь-3 в Сибирь,2022-04-23 03:00:00,2022-04-23,513.032,700.0
1,4000001,Казахстан - Сибирь-3 в Сибирь,2022-04-23 06:00:00,2022-04-23,421.376,700.0
2,4000001,Казахстан - Сибирь-3 в Сибирь,2022-04-23 08:00:00,2022-04-23,205.564,700.0
3,4000001,Казахстан - Сибирь-3 в Сибирь,2022-04-23 09:00:00,2022-04-23,182.247,700.0
4,4000001,Казахстан - Сибирь-3 в Сибирь,2022-04-23 11:00:00,2022-04-23,156.033,700.0


In [5]:
qwerty = f'SELECT *\
           FROM ComDis.TemperatureTown WHERE Date>=convert (datetime,\'{dataStart} 0:00:00\')'  

GtpPlanValue = pd.read_sql(qwerty, conn)
display(GtpPlanValue.head())

,Date,TownId,TownName,TemperatureProposal,TemperatureFact
0,2022-04-23,1900005,Черепаново,-2.86,NaN
1,2022-04-23,1900038,Береговой,-1.20,NaN
2,2022-04-23,1900060,Нижняя Омка,-2.40,NaN
3,2022-04-23,1900064,Оконешниково,-1.35,NaN
4,2022-04-23,1900081,Барабинск,-2.13,NaN


# РСВ

In [6]:
%%time

# Забираю 2 ценовые зоны одним запросом
qwerty = f'SELECT  SUM(Value) AS ValueSum, RegionName,PriceZone,Date \
           FROM ComDis.GtpPlanValue WHERE Date>=convert (datetime,\'{dataStart} 0:00:00\') \
           GROUP BY PriceZone, RegionName, Date \
           ORDER BY RegionName,Date'  

GtpPlanValue = pd.read_sql(qwerty, conn)
display(GtpPlanValue.head())

,ValueSum,RegionName,PriceZone,Date
0,914.335,Алтайский край,Сибирь,2022-04-23 00:00:00
1,928.778,Алтайский край,Сибирь,2022-04-23 01:00:00
2,986.123,Алтайский край,Сибирь,2022-04-23 02:00:00
3,1069.440,Алтайский край,Сибирь,2022-04-23 03:00:00
4,1170.740,Алтайский край,Сибирь,2022-04-23 04:00:00


CPU times: user 234 ms, sys: 11.5 ms, total: 245 ms
Wall time: 50.9 s


In [7]:
# Выделим час, день, месяц, год из даты
GtpPlanValue['hour'] = GtpPlanValue['Date'].dt.hour.astype(int)
GtpPlanValue['N года'] = GtpPlanValue['Date'].dt.year
GtpPlanValue['N месяца'] = GtpPlanValue['Date'].dt.month
GtpPlanValue['N дня'] = GtpPlanValue['Date'].dt.day
GtpPlanValue.head()

,ValueSum,RegionName,PriceZone,Date,hour,N года,N месяца,N дня
0,914.335,Алтайский край,Сибирь,2022-04-23 00:00:00,0,2022,4,23
1,928.778,Алтайский край,Сибирь,2022-04-23 01:00:00,1,2022,4,23
2,986.123,Алтайский край,Сибирь,2022-04-23 02:00:00,2,2022,4,23
3,1069.440,Алтайский край,Сибирь,2022-04-23 03:00:00,3,2022,4,23
4,1170.740,Алтайский край,Сибирь,2022-04-23 04:00:00,4,2022,4,23


In [8]:
# Сформируем сводную таблицу
GtpPlanPivot = pd.pivot_table(data=GtpPlanValue,
                              values='ValueSum',
                              index=['N года', 'N месяца', 'N дня'],
                              columns=['PriceZone', 'RegionName', 'hour'])
GtpPlanPivot.head()

PriceZone                           Европа                             \
RegionName            Астраханская область                              
hour                                    0        1        2        3    
N года N месяца N дня                                                   
2022   4        23                 362.750  354.445  351.445  350.432   
                24                 368.843  360.634  358.521  357.957   
                25                 361.741  353.331  352.583  354.812   
                26                 355.508  351.288  350.183  351.471   
                27                 353.418  344.918  342.782  343.343   

PriceZone                                                                    \
RegionName                                                                    
hour                        4        5        6        7        8        9    
N года N месяца N дня                                                         
2022   4        23     356.395  365.531  381.396  405.478  420.932  424.721   
                24     362.167  371.023  386.853  410.611  427.067  432.394   
                25     366.890  389.757  412.019  432.137  444.331  445.336   
                26     364.032  387.746  412.447  431.387  440.481  440.012   
                27     345.408  358.827  380.429  406.919  423.157  430.343   

PriceZone              ...          Сибирь                             \
RegionName             ... Томская область                              
hour                   ...              14       15       16       17   
N года N месяца N дня  ...                                              
2022   4        23     ...         845.004  840.583  844.687  851.705   
                24     ...         818.062  822.646  825.958  830.974   
                25     ...         850.370  852.050  854.130  847.319   
                26     ...         834.637  832.366  836.387  842.910   
                27     ...         812.504  814.113  820.831  824.046   

PriceZone                                                                    
RegionName                                                                   
hour                        18       19       20       21       22       23  
N года N месяца N дня                                                        
2022   4        23     826.357  793.641  761.836  741.741  729.261  722.530  
                24     802.704  776.456  743.651  727.514  715.178  709.492  
                25     817.335  778.275  743.674  724.342  712.386  707.425  
                26     809.737  770.073  735.682  717.192  705.995  699.740  
                27     799.532  761.600  728.656  710.200  698.873  692.802  

[5 rows x 1656 columns]

In [9]:
# Сохраним результаты
for x,cz in enumerate(['Европа', 'Сибирь']):
    GtpPlanPivot.loc[:, cz].reset_index().to_pickle(f'../models/data_RSV_pred_4_0-{x+1}cz.pkl')
    display(GtpPlanPivot.loc[:, cz].reset_index().head())

RegionName N года N месяца N дня Астраханская область                    \
hour                                                0        1        2   
0            2022        4    23              362.750  354.445  351.445   
1            2022        4    24              368.843  360.634  358.521   
2            2022        4    25              361.741  353.331  352.583   
3            2022        4    26              355.508  351.288  350.183   
4            2022        4    27              353.418  344.918  342.782   

RegionName                                      ... город Севастополь  \
hour              3        4        5        6  ...                14   
0           350.432  356.395  365.531  381.396  ...           187.743   
1           357.957  362.167  371.023  386.853  ...           187.588   
2           354.812  366.890  389.757  412.019  ...           198.393   
3           351.471  364.032  387.746  412.447  ...           178.565   
4           343.343  345.408  358.827  380.429  ...           177.812   

RegionName                                                                 \
hour             15       16       17       18       19       20       21   
0           183.793  182.722  183.681  187.604  199.048  214.937  213.241   
1           185.630  185.584  187.480  192.311  206.822  212.503  213.772   
2           197.308  196.365  196.268  198.942  212.012  228.978  229.231   
3           175.638  175.233  172.730  173.428  179.862  200.644  201.002   
4           174.798  171.694  170.526  171.329  177.762  192.235  192.502   

RegionName                    
hour             22       23  
0           201.438  184.650  
1           199.288  180.605  
2           213.251  190.565  
3           189.637  169.811  
4           175.635  154.513  

[5 rows x 1371 columns]

RegionName N года N месяца N дня Алтайский край                               \
hour                                          0        1         2         3   
0            2022        4    23        914.335  928.778   986.123  1069.440   
1            2022        4    24        913.222  922.004   966.503  1020.962   
2            2022        4    25        903.601  932.503  1027.920  1154.986   
3            2022        4    26        903.877  933.206  1026.800  1149.173   
4            2022        4    27        884.825  919.598  1011.078  1127.562   

RegionName                                ... Томская область           \
hour               4         5         6  ...              14       15   
0           1170.740  1240.694  1260.233  ...         845.004  840.583   
1           1107.660  1177.756  1204.039  ...         818.062  822.646   
2           1253.429  1305.875  1311.615  ...         850.370  852.050   
3           1243.109  1285.971  1279.443  ...         834.637  832.366   
4           1215.148  1261.889  1253.200  ...         812.504  814.113   

RegionName                                                                 \
hour             16       17       18       19       20       21       22   
0           844.687  851.705  826.357  793.641  761.836  741.741  729.261   
1           825.958  830.974  802.704  776.456  743.651  727.514  715.178   
2           854.130  847.319  817.335  778.275  743.674  724.342  712.386   
3           836.387  842.910  809.737  770.073  735.682  717.192  705.995   
4           820.831  824.046  799.532  761.600  728.656  710.200  698.873   

RegionName           
hour             23  
0           722.530  
1           709.492  
2           707.425  
3           699.740  
4           692.802  

[5 rows x 291 columns]

# ВСВГО

In [10]:
%%time

# Забираю 2 ценовые зоны одним запросом
qwerty = f'SELECT  Value AS ValueSum, RegionName,PriceZone,Date \
           FROM ComDis.RegionConsumeForecast WHERE Date>=convert (datetime,\'{dataStart} 0:00:00\') \
           ORDER BY RegionName,Date'  

RegionConsumeForecast = pd.read_sql(qwerty, conn)
display(RegionConsumeForecast.head())

,ValueSum,RegionName,PriceZone,Date
0,992.162339,Алтайский край,Сибирь,2022-04-23 00:00:00
1,1030.172598,Алтайский край,Сибирь,2022-04-23 01:00:00
2,1104.824258,Алтайский край,Сибирь,2022-04-23 02:00:00
3,1201.153262,Алтайский край,Сибирь,2022-04-23 03:00:00
4,1312.328126,Алтайский край,Сибирь,2022-04-23 04:00:00


CPU times: user 110 ms, sys: 8.79 ms, total: 119 ms
Wall time: 825 ms


In [11]:
# Выделим час, день, месяц, год из даты
RegionConsumeForecast['hour'] = RegionConsumeForecast['Date'].dt.hour.astype(int)
RegionConsumeForecast['N года'] = RegionConsumeForecast['Date'].dt.year
RegionConsumeForecast['N месяца'] = RegionConsumeForecast['Date'].dt.month
RegionConsumeForecast['N дня'] = RegionConsumeForecast['Date'].dt.day
RegionConsumeForecast.head()

,ValueSum,RegionName,PriceZone,Date,hour,N года,N месяца,N дня
0,992.162339,Алтайский край,Сибирь,2022-04-23 00:00:00,0,2022,4,23
1,1030.172598,Алтайский край,Сибирь,2022-04-23 01:00:00,1,2022,4,23
2,1104.824258,Алтайский край,Сибирь,2022-04-23 02:00:00,2,2022,4,23
3,1201.153262,Алтайский край,Сибирь,2022-04-23 03:00:00,3,2022,4,23
4,1312.328126,Алтайский край,Сибирь,2022-04-23 04:00:00,4,2022,4,23


In [12]:
# Сформируем сводную таблицу
RegionConsumePivot = pd.pivot_table(data=RegionConsumeForecast,
                                    values='ValueSum',
                                    index=['N года', 'N месяца', 'N дня'],
                                    columns=['PriceZone', 'RegionName', 'hour'])
RegionConsumePivot.head()

PriceZone             2 синхронная зона                            \
RegionName             Амурская область                             
hour                                 0            1            2    
N года N месяца N дня                                               
2022   4        23          1083.883403  1071.433783  1057.249257   
                24          1067.357218  1043.473822  1034.284853   
                25          1022.390893  1010.615308  1006.691263   
                26          1080.036694  1054.771495  1039.398580   
                27          1030.163420  1012.276458   997.611789   

PriceZone                                                                  \
RegionName                                                                  
hour                            3            4            5            6    
N года N месяца N дня                                                       
2022   4        23     1060.135527  1063.356956  1068.641809  1087.154607   
                24     1032.062414  1035.604286  1045.731741  1060.892797   
                25     1003.689402  1010.390323  1022.331091  1042.342100   
                26     1030.601142  1032.116319  1042.656022  1057.229224   
                27      998.419554  1003.440557  1019.694875  1037.092560   

PriceZone                                                     ...  \
RegionName                                                    ...   
hour                            7            8            9   ...   
N года N месяца N дня                                         ...   
2022   4        23     1119.625694  1142.217783  1163.150477  ...   
                24     1087.289938  1111.973239  1126.605687  ...   
                25     1080.210127  1120.619707  1140.021985  ...   
                26     1080.388979  1103.723919  1118.429490  ...   
                27     1061.076741  1075.528439  1087.402995  ...   

PriceZone                      Сибирь                                       \
RegionName            Томская область                                        
hour                               14          15           16          17   
N года N месяца N дня                                                        
2022   4        23         955.987030  955.078103   970.832222  965.907908   
                24         971.329628  967.233880   983.348186  973.094230   
                25         994.727354  994.033393  1004.838431  992.309543   
                26         978.367054  982.606834   995.207672  989.011085   
                27         961.477475  958.061734   968.293517  965.632437   

PriceZone                                                              \
RegionName                                                              
hour                           18          19          20          21   
N года N месяца N дня                                                   
2022   4        23     930.123138  886.676338  857.460828  840.794318   
                24     929.283590  885.327449  858.421065  843.782387   
                25     948.295130  901.770025  870.354738  856.490499   
                26     942.630248  891.653195  855.964776  841.505243   
                27     923.419260  872.902974  844.833834  828.502798   

PriceZone                                      
RegionName                                     
hour                           22          23  
N года N месяца N дня                          
2022   4        23     833.245777  825.211888  
                24     836.261976  832.717816  
                25     847.197726  842.961475  
                26     838.479122  833.019064  
                27     816.790090  814.335591  

[5 rows x 1728 columns]

In [13]:
# Сохраним результаты
for x,cz in enumerate(['Европа', 'Сибирь']):
    RegionConsumePivot.loc[:, cz].reset_index().to_pickle(f'../models/data_VSVGO_pred_4_0-{x+1}cz.pkl')
    display(RegionConsumePivot.loc[:, cz].reset_index().head())

RegionName N года N месяца N дня Астраханская область                          \
hour                                                0           1           2   
0            2022        4    23           346.995398  340.075973  338.076888   
1            2022        4    24           359.524456  351.179601  349.156307   
2            2022        4    25           337.553185  331.052694  329.890071   
3            2022        4    26           333.247864  328.436205  327.774612   
4            2022        4    27           321.577930  315.442283  314.201532   

RegionName                                                  ...  \
hour                 3           4           5           6  ...   
0           339.808437  345.113241  366.932235  388.345921  ...   
1           351.156111  352.187759  363.561322  380.982227  ...   
2           336.571922  352.305942  391.154626  402.728861  ...   
3           332.418376  343.902898  376.767367  395.153114  ...   
4           318.421969  329.361383  360.277221  374.636571  ...   

RegionName Ярославская область                                      \
hour                        14          15          16          17   
0                   869.972807  872.219743  859.423522  872.119678   
1                   833.972912  828.743052  827.636913  842.939949   
2                   939.597908  925.322242  890.872620  892.046372   
3                   977.323697  957.416637  932.615940  927.216219   
4                  1000.185644  976.537566  949.921863  953.459146   

RegionName                                                              \
hour                18          19          20          21          22   
0           882.319093  892.512752  912.574126  865.894007  817.615090   
1           853.396919  867.804043  891.174860  862.960998  813.320416   
2           888.103464  902.593049  948.471804  860.585396  813.478635   
3           913.369296  917.956157  946.201177  894.129080  830.852407   
4           927.700442  953.655838  970.448357  906.568955  840.647137   

RegionName              
hour                23  
0           791.521229  
1           788.286942  
2           782.669042  
3           796.181692  
4           802.956998  

[5 rows x 1299 columns]

RegionName N года N месяца N дня Алтайский край                            \
hour                                          0            1            2   
0            2022        4    23     992.162339  1030.172598  1104.824258   
1            2022        4    24     959.177829   976.594720  1056.954434   
2            2022        4    25    1011.270924  1077.162506  1213.934874   
3            2022        4    26     999.646304  1041.684480  1187.629147   
4            2022        4    27     957.774203  1016.114866  1145.577126   

RegionName                                                      ...  \
hour                  3            4            5            6  ...   
0           1201.153262  1312.328126  1360.507312  1373.332115  ...   
1           1141.257160  1247.628462  1294.443792  1305.098677  ...   
2           1344.031825  1421.797595  1432.227047  1410.516313  ...   
3           1332.391152  1402.646946  1414.837800  1407.976090  ...   
4           1279.729167  1363.524871  1383.409477  1382.786807  ...   

RegionName Томская область                                                   \
hour                    14          15           16          17          18   
0               955.987030  955.078103   970.832222  965.907908  930.123138   
1               971.329628  967.233880   983.348186  973.094230  929.283590   
2               994.727354  994.033393  1004.838431  992.309543  948.295130   
3               978.367054  982.606834   995.207672  989.011085  942.630248   
4               961.477475  958.061734   968.293517  965.632437  923.419260   

RegionName                                                              
hour                19          20          21          22          23  
0           886.676338  857.460828  840.794318  833.245777  825.211888  
1           885.327449  858.421065  843.782387  836.261976  832.717816  
2           901.770025  870.354738  856.490499  847.197726  842.961475  
3           891.653195  855.964776  841.505243  838.479122  833.019064  
4           872.902974  844.833834  828.502798  816.790090  814.335591  

[5 rows x 267 columns]